In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine
from functions.pubgapi import get_tournament_info
from functions.pubgapi import get_match_info
from functions.pubgapi import get_match_participant
from functions.pubgapi import get_match_participant_single
from functions.pubgapi import check_missing_value

# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

# DB connect

# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn = engine.connect()

In [2]:
# Tournament id 값 조회
tournament_info = get_tournament_info(api_key)
check_missing_value("tournament_info",tournament_info)
tournament_id = tournament_info["id"]
num_tournament = len(tournament_id)

print(f"{num_tournament} 개의 tournament_id 조회 완료")

# Tournament info 테이블 저장(local)
tournament_info.to_csv(f"./DB/tournament_info.csv")

# Tournament info 테이블 create
tournament_info.to_sql(name="tournament_info", con=conn, if_exists='replace', index=False)
print(f"{num_tournament} 개의 tournament_id 저장 완료\n")

469 개의 tournament_id 조회 완료
469 개의 tournament_id 저장 완료



In [4]:
# 각 tournament 별 match 정보 조회
for tournament_index in range(10,20):
    cur_match_info = get_match_info(api_key, f"{tournament_id[tournament_index]}")
    cur_match_info.insert(loc=0, column="tournament_id", value=f"{tournament_id[tournament_index]}")

    if os.path.isfile("./DB/match_info.csv"):
        match_info = pd.read_csv(f"./DB/match_info.csv")
        match_info.drop(["Unnamed: 0"], axis = 1, inplace = True)
        match_info = pd.concat([match_info, cur_match_info], ignore_index=True)
        match_info.to_csv(f"./DB/match_info.csv")
    else:
        # 해당 경로에 match_info.csv 파일이 없으면, 파일 생성
        cur_match_info.to_csv(f"./DB/match_info.csv")

In [ ]:
# 각 tournament의 Match info 테이블 조회
loading_bar = {0:'-', 1:'\\', 2:'|', 3:'/'}
for tournament_index in range(0,9): #len(tournament_info["id"])
    # index 설정 및 해당 인덱스의 tournament_id 값 불러오기
    tournament_name = tournament_info["id"][tournament_index]
    tournament_createdAt = tournament_info["createdAt"][tournament_index]

    # cur_match_info(현재 match info) 불러오기
    cur_match_info = get_match_info(api_key, tournament_name)

    # match_participant_single 불러오기
    for match_index in range(len(cur_match_info["matchId"])):
        match_participant_single = get_match_participant_single(api_key, cur_match_info["matchId"][match_index])
        check_missing_value(cur_match_info["matchId"][match_index], match_participant_single)

        # Z-score normalization 수행
        # data = z_normalization(match_participant_single)
        data = match_participant_single

        if os.path.isfile("./Data/Train_data/train_match_data.csv"):
            # 해당 경로에 train_match_data.csv 파일이 있으면, 현재 내용을 해당 파일에 추가
            train_data = pd.read_csv(f"./Data/Train_data/train_match_data.csv")
            train_data.drop(["Unnamed: 0"], axis = 1, inplace = True)
            train_data = pd.concat([train_data, data], ignore_index=True)
            train_data.to_csv(f"./Data/Train_data/train_match_data.csv")
        else:
            # 해당 경로에 train_match_data.csv 파일이 없으면, 파일 생성
            data.to_csv(f"./Data/Train_data/train_match_data.csv")
        print(f"\rLoading tournament data {loading_bar[match_index%4]}", end="")

In [ ]:
# Save match_info to csv file
for idx in range(0, 1):
    # idx를 지정하여 해당 위치의 id값 출력
    tournament_index_id = tournament_id[idx]
    cur_match_info = get_match_info(api_key, tournament_index_id)
    num_match = len(cur_match_info["matchId"])

    match_info_id = cur_match_info["matchId"]
    match_participant = get_match_participant(api_key, match_info_id)
    match_participant.to_csv(f"./Data/{tournament_index_id}_match_info.csv")
    print(f"Tournament name {idx}: {tournament_index_id} ({num_match} matches)")
print("complete")

# Match participant 조회
idx2 = 2
match_info_id = cur_match_info["matchId"]
match_info_id_single = match_info_id[idx2]
match_participant = get_match_participant_single(api_key, match_info_id_single)

# AWS RDS에 테이블 create

if match_participant["kills"].sum() != 0:
    # Kill 수의 합이 0인 경우 제외(연습게임으로 간주)
    match_participant.to_sql(name="train_match_data", con=conn, if_exists='append', index=False)

# 업로드된 테이블 확인
sql = "SHOW TABLES;"
result = pd.read_sql(sql, conn)
print(sql)

# 내용 확인(SELECT *)
sql = "SELECT * FROM train_match_data;"
result1 = pd.read_sql(sql, conn)
print(sql)

# db접속 종료
conn.close()